In [1]:
from Loader import fileloader as fl
from Loader import datavalidator as dv
from Loader import datahandledatabase as dhdb
import posixpath

month = '201811'
inifile = fl.ReadIniFile()
defaultpath = inifile.getDefaultPath()
testpath = inifile.getTestPath()

Setting defaultpath as D:/Datos de Usuario/jsaenza/Documents/Comisiones/MercadoEmpresas/Data Fuente Comisiones/xlsx
Setting testpath as D:/Datos de Usuario/jsaenza/Documents/Comisiones/MercadoEmpresas/Data Fuente Comisiones/test


In [2]:
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [11]:
# Importando la información
# Validar en los equipos quincenales la jerarquia de ventas
# Copiar los reportes comisionales del periodo : Grandes Cuentas y Pymes

# Inar y eliminando las migraciones


#inarbruto = loader.loadFile('Inar_bruto')
#jerarquia = loader.loadFile('Jerarquia')
comisionantes = loader.loadFile('Comisionantes_voz')

dbparser = inifile.getDbIniFileParser()

dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)
tblempleados = dbmanager.loadData('tblEmpleados')

El tamaño de Comisionantes_voz es 308 registros
Los registros de la tabla tblEmpleados es 1049 registros  


In [12]:
# Armando las validaciones

valinarfl = {'inarbruto' : inarbruto, 'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valinarobj = dv.ValidateInar(valinarfl)
inarvalidation = valinarobj.validation()


valmultipledatafl = {'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valmultipledataobj = dv.ValidateMultipleData(valmultipledatafl)
multipledatavalidation = valmultipledataobj.validation()

#Exportando validaciones

xlsxfile1 = posixpath.join(testpath, month + '_Validaciones en INAR.xlsx')
exportparams = {'xlsxfile' : xlsxfile1, 'dataframe' : inarvalidation}
valinarobj.exportValidation(exportparams)

#"""
xlsxfile2 = posixpath.join(testpath, month + '_Otras Validaciones.xlsx')
exportparams = {'xlsxfile' : xlsxfile2, 'dataframe' : multipledatavalidation}
valmultipledataobj.exportValidation(exportparams)
#"""

D:\Datos de Usuario\jsaenza\Documents\Comisiones\MercadoEmpresas\Comisiones-python\Loader\datavalidator.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  blanksdoc['OBSERVACION'] = 'blanks en documento'
D:\Datos de Usuario\jsaenza\Documents\Comisiones\MercadoEmpresas\Comisiones-python\Loader\datavalidator.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  conjerarquia['CONCOLSJER'] = conjerarquia.apply(lambda x : '|'.join(x), axis = 1)


Archivo exportado D:/Datos de Usuario/jsaenza/Documents/Comisiones/MercadoEmpresas/Data Fuente Comisiones/test/201811_Validaciones en INAR.xlsx con 120 registros
Archivo exportado D:/Datos de Usuario/jsaenza/Documents/Comisiones/MercadoEmpresas/Data Fuente Comisiones/test/201811_Otras Validaciones.xlsx con 41 registros


In [6]:
# Detectando los registros de Tienda que no tienen Orden
str_tienda = 'Tienda'
lst_estados = ['NEWS','ADDS','FCHURN','DEOF','REAC']

mask0 = inarbruto['ESTADO'].isin(lst_estados)
mask1 = inarbruto['NRO_ORDEN'].isnull()
inar_f1 = inarbruto[mask0 & mask1]
mask2 = inar_f1['VENDEDOR'].notnull()
inar_f2 = inar_f1[mask2]
mask3 = inar_f2['VENDEDOR'].str.contains(str_tienda)
inar_f3 = inar_f2[mask3]

#mask2 = inarbruto['VENDEDOR'].str.contains(str_tienda)
#inar_filtrado = inarbruto[mask2]



In [7]:
inar_f2.to_csv(posixpath.join(testpath, 'inar_sin_orden.csv'))

In [10]:
tblempleados[tblempleados['CODIGO_INAR'] == 'RPAZ2']

,ID_EMPL,DNI,NOMBRES,APELLIDO_PATERNO,APELLIDO_MATERNO,POSICION_EMPL,FECHA_INGRESO,FECHA_ACTUALIZACION,PERIODO_ACTIVACION,CODIGO_INAR,FECHA_CESE
817,4482,44609011,RICHARD,PAZ LOAYZA,,EJECUTIVO DE CORPORACIONES,2018-11-01 00:00:00,2017-11-02 09:08:42,201710,RPAZ2,


In [8]:
tblempleados.dtypes

ID_EMPL                 int64
DNI                    object
NOMBRES                object
APELLIDO_PATERNO       object
APELLIDO_MATERNO       object
POSICION_EMPL          object
FECHA_INGRESO          object
FECHA_ACTUALIZACION    object
PERIODO_ACTIVACION      int64
CODIGO_INAR            object
FECHA_CESE             object
dtype: object